### ENVIRONMENT

In [7]:
"""
This is a gym environment based for the drone_game
The goal is to reach randomly positoned targets
"""

import os
from math import sin, cos, pi, sqrt
from random import randrange

import numpy as np
import gym
from gym import spaces

import pygame
from pygame.locals import *


class droneEnv(gym.Env):
    def __init__(self, render_every_frame, mouse_target):
        super(droneEnv, self).__init__()

        self.render_every_frame = render_every_frame
        # Makes the target follow the mouse
        self.mouse_target = mouse_target

        # Initialize Pygame, load sprites
        pygame.init()
        self.screen = pygame.display.set_mode((800, 800))
        self.FramePerSec = pygame.time.Clock()

        self.player = pygame.image.load(os.path.join("assets/sprites/drone_old.png"))
        self.player.convert()

        self.target = pygame.image.load(os.path.join("assets/sprites/target_old.png"))
        self.target.convert()

        pygame.font.init()
        self.myfont = pygame.font.SysFont("Comic Sans MS", 20)

        # Physics constants
        self.FPS = 60
        self.gravity = 0.08
        self.thruster_amplitude = 0.04
        self.diff_amplitude = 0.003
        self.thruster_mean = 0.04
        self.mass = 1
        self.arm = 25

        # Initialize variables
        (self.a, self.ad, self.add) = (0, 0, 0)
        (self.x, self.xd, self.xdd) = (400, 0, 0)
        (self.y, self.yd, self.ydd) = (400, 0, 0)
        self.xt = randrange(200, 600)
        self.yt = randrange(200, 600)

        # Initialize game variables
        self.target_counter = 0
        self.reward = 0
        self.time = 0
        self.time_limit = 20
        if self.mouse_target is True:
            self.time_limit = 1000

        # 2 action thrust amplitude and thrust difference in float values between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,))
        # 6 observations: angle_to_up, velocity, angle_velocity, distance_to_target, angle_to_target, angle_target_and_velocity
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,))

    def reset(self):
        # Reset variables
        (self.a, self.ad, self.add) = (0, 0, 0)
        (self.x, self.xd, self.xdd) = (400, 0, 0)
        (self.y, self.yd, self.ydd) = (400, 0, 0)
        self.xt = randrange(200, 600)
        self.yt = randrange(200, 600)

        self.target_counter = 0
        self.reward = 0
        self.time = 0

        return self.get_obs()

    def get_obs(self) -> np.ndarray:
        """
        Calculates the observations

        Returns:
            np.ndarray: The normalized observations:
            - angle_to_up : angle between the drone and the up vector (to observe gravity)
            - velocity : velocity of the drone
            - angle_velocity : angle of the velocity vector
            - distance_to_target : distance to the target
            - angle_to_target : angle between the drone and the target
            - angle_target_and_velocity : angle between the to_target vector and the velocity vector
        """
        angle_to_up = self.a / 180 * pi
        velocity = sqrt(self.xd**2 + self.yd**2)
        angle_velocity = self.ad
        distance_to_target = (
            sqrt((self.xt - self.x) ** 2 + (self.yt - self.y) ** 2) / 500
        )
        angle_to_target = np.arctan2(self.yt - self.y, self.xt - self.x)
        # Angle between the to_target vector and the velocity vector
        angle_target_and_velocity = np.arctan2(
            self.yt - self.y, self.xt - self.x
        ) - np.arctan2(self.yd, self.xd)
        distance_to_target = (
            sqrt((self.xt - self.x) ** 2 + (self.yt - self.y) ** 2) / 500
        )
        return np.array(
            [
                angle_to_up,
                velocity,
                angle_velocity,
                distance_to_target,
                angle_to_target,
                angle_target_and_velocity,
            ]
        ).astype(np.float32)

    def step(self, action):
        # Game loop
        self.reward = 0.0
        (action0, action1) = (action[0], action[1])

        # Act every 5 frames
        for _ in range(5):
            self.time += 1 / 60

            if self.mouse_target is True:
                self.xt, self.yt = pygame.mouse.get_pos()

            # Initialize accelerations
            self.xdd = 0
            self.ydd = self.gravity
            self.add = 0
            thruster_left = self.thruster_mean
            thruster_right = self.thruster_mean

            thruster_left += action0 * self.thruster_amplitude
            thruster_right += action0 * self.thruster_amplitude
            thruster_left += action1 * self.diff_amplitude
            thruster_right -= action1 * self.diff_amplitude

            # Calculating accelerations with Newton's laws of motions
            self.xdd += (
                -(thruster_left + thruster_right) * sin(self.a * pi / 180) / self.mass
            )
            self.ydd += (
                -(thruster_left + thruster_right) * cos(self.a * pi / 180) / self.mass
            )
            self.add += self.arm * (thruster_right - thruster_left) / self.mass

            self.xd += self.xdd
            self.yd += self.ydd
            self.ad += self.add
            self.x += self.xd
            self.y += self.yd
            self.a += self.ad

            dist = sqrt((self.x - self.xt) ** 2 + (self.y - self.yt) ** 2)

            # Reward per step survived
            self.reward += 1 / 60
            # Penalty according to the distance to target
            self.reward -= dist / (100 * 60)

            if dist < 50:
                # Reward if close to target
                self.xt = randrange(200, 600)
                self.yt = randrange(200, 600)
                self.reward += 100
                
                
                #
                self.target_counter+=1

            # If out of time
            if self.time > self.time_limit:
                done = True
                break

            # If too far from target (crash)
            elif dist > 1000:
                self.reward -= 1000
                done = True
                break

            else:
                done = False

            if self.render_every_frame is True:
                self.render("yes")

        info = {}

        return (
            self.get_obs(),
            self.reward,
            done,
            info,
        )

    def render(self, mode):
        # Pygame rendering
        pygame.event.get()
        self.screen.fill(0)
        self.screen.blit(
            self.target,
            (
                self.xt - int(self.target.get_width() / 2),
                self.yt - int(self.target.get_height() / 2),
            ),
        )
        player_copy = pygame.transform.rotate(self.player, self.a)
        self.screen.blit(
            player_copy,
            (
                self.x - int(player_copy.get_width() / 2),
                self.y - int(player_copy.get_height() / 2),
            ),
        )

        textsurface = self.myfont.render(
            "Collected: " + str(self.target_counter), False, (255, 255, 255)
        )
        self.screen.blit(textsurface, (20, 20))
        textsurface3 = self.myfont.render(
            "Time: " + str(int(self.time)), False, (255, 255, 255)
        )
        self.screen.blit(textsurface3, (20, 50))

        pygame.display.update()
        self.FramePerSec.tick(self.FPS)

    def close(self):
        pass


## TEST ENVIRONMENT WORKING

In [2]:
env = droneEnv(True, False)

obs = env.reset()
env.render("yes")

print("Observation space:")
print(env.observation_space)
print("")
print("Action space:")
print(env.action_space)
print("")
print("Action space sample:")
print(env.action_space.sample())

# Choose an action to execute n_steps times
action = np.array([0.0, 1.0])
n_steps = 10
for step in range(n_steps):
    print("Step {}".format(step + 1))
    obs, reward, done, info = env.step(action)
    print("obs=", obs, "reward=", reward, "done=", done)
    env.render("yes")
    if done:
        print("Done!", "reward=", reward)
        break


Observation space:
Box([-inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf], (6,), float32)

Action space:
Box([-1. -1.], [1. 1.], (2,), float32)

Action space sample:
[ 0.39743373 -0.84471196]
Step 1
obs= [-0.03926991  0.00418868 -0.75        0.12176462  2.4027543   2.3931987 ] reward= 0.03260112590763964 done= False
Step 2
obs= [-0.14398967  0.03454228 -1.5         0.12190311  2.4038532   2.3648481 ] reward= 0.03257146751646054 done= False
Step 3
obs= [-0.31415927  0.1170206  -2.25        0.12244841  2.4085126   2.3204532 ] reward= 0.03242365841406434 done= False
Step 4
obs= [-0.5497787   0.27655566 -3.          0.12379135  2.421157    2.2645454 ] reward= 0.032011134430318694 done= False
Step 5
obs= [-0.850848    0.53502405 -3.75        0.12635559  2.4480414   2.2036622 ] reward= 0.031160051441842983 done= False
Step 6
obs= [-1.2173672   0.907548   -4.5         0.13052404  2.4964764   2.1457124 ] reward= 0.029702167775983026 done= False
Step 7
obs= [-1.6493361  1.3969848 -5.25 

## TRAIN SAC AGENT

In [6]:
from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback
import wandb
from wandb.integration.sb3 import WandbCallback


run = wandb.init(
    project="CSP",
    sync_tensorboard=True,
    monitor_gym=True,
)

# Create log dir
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)

# Create and wrap the environment
env = droneEnv(False, False)
env = Monitor(env, log_dir)

# Create SAC agent
model = SAC("MlpPolicy", env, verbose=1, tensorboard_log=log_dir)

training_steps = 1_000_000


# Create checkpoint callback
checkpoint_callback = CheckpointCallback(
    save_freq=training_steps, save_path=log_dir, name_prefix="sac_model_v5"
)

# Train the agent
model.learn(
    total_timesteps=training_steps,
    callback=[
        checkpoint_callback,
        WandbCallback(
            gradient_save_freq=training_steps,
            model_save_path=f"models/{run.id}",
            model_save_freq=training_steps,
            verbose=2,
        ),
    ],
)

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Using cpu device
Wrapping the env in a DummyVecEnv.
Logging to tmp/SAC_4
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 41.8      |
|    ep_rew_mean     | -1.01e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 145       |
|    time_elapsed    | 1         |
|    total_timesteps | 167       |
| train/             |           |
|    actor_loss      | 13.2      |
|    critic_loss     | 1.48e+04  |
|    ent_coef        | 0.981     |
|    ent_coef_loss   | -0.0654   |
|    learning_rate   | 0.0003    |
|    n_updates       | 66        |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 39.5      |
|    ep_rew_mean     | -1.01e+03 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 90        |
|    time_elapsed    | 3         |
|    total_timesteps | 316       |
| train/         

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 35.7     |
|    ep_rew_mean     | -994     |
| time/              |          |
|    episodes        | 60       |
|    fps             | 59       |
|    time_elapsed    | 35       |
|    total_timesteps | 2140     |
| train/             |          |
|    actor_loss      | 233      |
|    critic_loss     | 2.52e+03 |
|    ent_coef        | 0.888    |
|    ent_coef_loss   | -0.00835 |
|    learning_rate   | 0.0003   |
|    n_updates       | 2039     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 35.3     |
|    ep_rew_mean     | -995     |
| time/              |          |
|    episodes        | 64       |
|    fps             | 59       |
|    time_elapsed    | 38       |
|    total_timesteps | 2260     |
| train/             |          |
|    actor_loss      | 257      |
|    critic_loss     | 1.32e+03 |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 34       |
|    ep_rew_mean     | -996     |
| time/              |          |
|    episodes        | 120      |
|    fps             | 56       |
|    time_elapsed    | 73       |
|    total_timesteps | 4175     |
| train/             |          |
|    actor_loss      | 376      |
|    critic_loss     | 757      |
|    ent_coef        | 0.938    |
|    ent_coef_loss   | -0.0431  |
|    learning_rate   | 0.0003   |
|    n_updates       | 4074     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 33.9     |
|    ep_rew_mean     | -995     |
| time/              |          |
|    episodes        | 124      |
|    fps             | 56       |
|    time_elapsed    | 76       |
|    total_timesteps | 4313     |
| train/             |          |
|    actor_loss      | 421      |
|    critic_loss     | 1.66e+03 |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 42.5     |
|    ep_rew_mean     | -983     |
| time/              |          |
|    episodes        | 180      |
|    fps             | 55       |
|    time_elapsed    | 127      |
|    total_timesteps | 7043     |
| train/             |          |
|    actor_loss      | 561      |
|    critic_loss     | 511      |
|    ent_coef        | 0.51     |
|    ent_coef_loss   | 0.08     |
|    learning_rate   | 0.0003   |
|    n_updates       | 6942     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 44.4     |
|    ep_rew_mean     | -983     |
| time/              |          |
|    episodes        | 184      |
|    fps             | 55       |
|    time_elapsed    | 133      |
|    total_timesteps | 7376     |
| train/             |          |
|    actor_loss      | 575      |
|    critic_loss     | 904      |
|    ent_coef 

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 940      |
| time/              |          |
|    episodes        | 4140     |
|    fps             | 46       |
|    time_elapsed    | 20222    |
|    total_timesteps | 943751   |
| train/             |          |
|    actor_loss      | -220     |
|    critic_loss     | 130      |
|    ent_coef        | 0.108    |
|    ent_coef_loss   | -0.0362  |
|    learning_rate   | 0.0003   |
|    n_updates       | 943650   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 944      |
| time/              |          |
|    episodes        | 4144     |
|    fps             | 46       |
|    time_elapsed    | 20241    |
|    total_timesteps | 944711   |
| train/             |          |
|    actor_loss      | -221     |
|    critic_loss     | 120      |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 960      |
| time/              |          |
|    episodes        | 4200     |
|    fps             | 46       |
|    time_elapsed    | 20514    |
|    total_timesteps | 958151   |
| train/             |          |
|    actor_loss      | -218     |
|    critic_loss     | 307      |
|    ent_coef        | 0.107    |
|    ent_coef_loss   | 0.255    |
|    learning_rate   | 0.0003   |
|    n_updates       | 958050   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 962      |
| time/              |          |
|    episodes        | 4204     |
|    fps             | 46       |
|    time_elapsed    | 20534    |
|    total_timesteps | 959111   |
| train/             |          |
|    actor_loss      | -234     |
|    critic_loss     | 30.5     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 947      |
| time/              |          |
|    episodes        | 4260     |
|    fps             | 46       |
|    time_elapsed    | 20801    |
|    total_timesteps | 972551   |
| train/             |          |
|    actor_loss      | -220     |
|    critic_loss     | 652      |
|    ent_coef        | 0.103    |
|    ent_coef_loss   | -0.0165  |
|    learning_rate   | 0.0003   |
|    n_updates       | 972450   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 945      |
| time/              |          |
|    episodes        | 4264     |
|    fps             | 46       |
|    time_elapsed    | 20819    |
|    total_timesteps | 973511   |
| train/             |          |
|    actor_loss      | -225     |
|    critic_loss     | 189      |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 962      |
| time/              |          |
|    episodes        | 4320     |
|    fps             | 46       |
|    time_elapsed    | 21080    |
|    total_timesteps | 986951   |
| train/             |          |
|    actor_loss      | -231     |
|    critic_loss     | 508      |
|    ent_coef        | 0.104    |
|    ent_coef_loss   | -0.265   |
|    learning_rate   | 0.0003   |
|    n_updates       | 986850   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 966      |
| time/              |          |
|    episodes        | 4324     |
|    fps             | 46       |
|    time_elapsed    | 21100    |
|    total_timesteps | 987911   |
| train/             |          |
|    actor_loss      | -233     |
|    critic_loss     | 347      |
|    ent_coef 

OSError: [WinError 1314] A required privilege is not held by the client: 'C:\\Users\\AVISH\\models\\nc0cqlss\\model.zip' -> 'C:\\Users\\AVISH\\wandb\\run-20231210_143604-nc0cqlss\\files\\model.zip'

## TEST TRAINED AGENT IN THE ENVIRONMENT

In [ ]:
import torch as th
from stable_baselines3 import SAC
MODEL_PATH =r"C:\Users\AVISH\models\9ygbjh9i\model.zip"

# Create and wrap the environment
env = droneEnv(True, False)

# Load the trained agent
model = SAC.load(MODEL_PATH, env=env)

# Evaluate the agent
for i in range(5):
    obs = env.reset()
    done = False
    episode_reward = 0
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
    print("Episode reward", episode_reward)
    env.render("yes")

In [17]:
model.tau

0.005